# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core.model import Model
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import joblib
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
import math
from sklearn.model_selection import train_test_split


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
df_bonds = ws.datasets['df_bonds'] 
experiment=Experiment(ws, 'bonds_oas')

cluster_name='valeriy-cluster'

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [3]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.25)
'Explanation: if the current run is not exceeding in accuracy the best one by 25%, it will be cancelled. Good balance between efficiency and not cancelling many potentially good runs due to mis-estimation of the accuracy metric at the early calculation stage'
#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"alpha":uniform(0.1, 1),"l1_ratio":uniform(0.1,1)})
'The benefit of this sampler is efficiency. It allows one to randomly select a limited number of parameters, and the randomness ensures that a relatively large universe of parameters is covered.'
'Compared to the alteratives, i.e. grid sampling, it does not test all the parameters hence saves time.'
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(".",compute_target= cluster, entry_script='./bonds_hyperparameter.py')
'In the bonds_hyperparameter I am using a Ridge (Tikhonov) regression model. It is a form of a regression model where the loss function is the linear least squares function.'
'Since the dependent variable (bond spread) is continious numeric, this form of the mosel is appropriate to choose.'

hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=30)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [4]:
#TODO: Submit your experiment
remote_run=experiment.submit(hyperdrive_run_config,show_output=True)
remote_run.wait_for_completion()

{'runId': 'HD_ec57838e-e157-49a3-becf-2d14e0fe73e6',
 'target': 'valeriy-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-02-22T07:51:51.128509Z',
 'endTimeUtc': '2022-02-22T08:08:21.974088Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'db9b0ef1-0ca4-4f56-9969-250b4cb1f02b',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1068-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.38.0',
  'space_size': 'infinite_space_size',
  'score': '-3.95248673073119e+24',
  'best_child_run_id': 'HD_ec57838e-e157-49a3-becf-2d14e0fe73e6_28',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg187275.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_ec57838e-e157-49a3-be

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(remote_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [12]:
best_run=remote_run.get_best_run_by_primary_metric()
hyperparam_best_run=best_run.get_details()['runDefinition']['arguments']
print(hyperparam_best_run)

['--alpha', '0.7439226961534412', '--l1_ratio', '0.9545007230942518']


In [13]:
#TODO: Save the best model
model_to_save_regression=SGDRegressor(alpha=float(hyperparam_best_run[1]),l1_ratio=float(hyperparam_best_run[-1]))
joblib.dump(model_to_save_regression, 'valeriy_best_model_hyperparam.pkl')

['valeriy_best_model_hyperparam.pkl']

In [14]:
df_bonds = ws.datasets['df_bonds'] 
x=df_bonds.to_pandas_dataframe().drop('OAS',1)
y=df_bonds.to_pandas_dataframe()['OAS']    
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=10)
model_to_save_regression.fit(X=x_train, y=y_train)
y_pred_regression = model_to_save_regression.predict(X=x_test)
rmse_regression = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred_regression))
print('rmse, regression:',rmse_regression)

rmse, regression: 2709808860375311.0


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
model=Model.register(ws,model_name='valeriy_model_hyperparam',model_path='./valeriy_best_model_hyperparam.pkl',
description='This is a model using hyperparameter tuning. The accuracy of this model is lower than that of the automl. Hence this model will be registered but not deployed')

Registering model valeriy_model_hyperparam


TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

